# Museum Analysis

This notebook connects to the museum database and conducts simple analysis of the data.

## Imports

In [76]:
from os import environ
from dotenv import load_dotenv
import psycopg2
import psycopg2.extras
import psycopg2.extensions


load_dotenv(".env.prod")

True

## Setup

In [77]:
def get_connection():
    """Connect to database"""
    return psycopg2.connect(
        user=environ["DATABASE_USERNAME"],
        password=environ["DATABASE_PASSWORD"],
        host=environ["DATABASE_HOST"],
        port=environ["DATABASE_PORT"],
        database=environ["DATABASE_NAME"]
    )


def get_cursor(connection):
    """Returns cursor"""
    return connection.cursor(cursor_factory=psycopg2.extras.DictCursor)
conn = get_connection()
cursor = get_cursor(conn)


# # Queries

In [78]:

query = """SELECT exhibition_id, exhibition_name, exhibition_description, department_id, floor_id, exhibition_start_date
FROM exhibition;"""

cursor.execute(query)
response = cursor.fetchall()
print(response)


[[1, 'Adaptation', 'How insect evolution has kept pace with an industrialised world', 1, 4, datetime.date(2019, 7, 1)], [2, 'Measureless to Man', 'An immersive 3D experience: delve deep into a previously-inaccessible cave system.', 2, 1, datetime.date(2021, 8, 23)], [3, 'Thunder Lizards', 'How new research is making scientists rethink what dinosaurs really looked like.', 3, 1, datetime.date(2023, 2, 1)], [4, 'The Crenshaw Collection', 'An exhibition of 18th Century watercolours, mostly focused on South American wildlife.', 4, 2, datetime.date(2021, 3, 3)], [5, 'Our Polluted World', 'A hard-hitting exploration of humanity"s impact on the environment.', 5, 3, datetime.date(2021, 5, 12)], [6, 'Cetacean Sensations', 'Whales: from ancient myth to critically endangered.', 4, 1, datetime.date(2019, 7, 1)]]


**Count of Rating Interactions by Exhibition**: Counts how many rating interactions each exhibition has received.

In [79]:
query = """SELECT e.exhibition_name, COUNT(ra_i.rating_interaction_id) AS rating_count
FROM exhibition e
JOIN rating_interaction ra_i ON e.exhibition_id = ra_i.exhibition_id
GROUP BY e.exhibition_name
ORDER BY rating_count DESC;"""

cursor.execute(query)
response = cursor.fetchall()
print(response)

[['Our Polluted World', 10205], ['Cetacean Sensations', 9969], ['The Crenshaw Collection', 8733], ['Measureless to Man', 8370], ['Thunder Lizards', 7184], ['Adaptation', 5862]]


**Count of Request Interactions by Exhibition**: Shows how many request interactions (e.g., assistance or emergency) each exhibition has received.

In [80]:
query = """SELECT e.exhibition_name, COUNT(ra_i.request_interaction_id) AS request_count
FROM exhibition e
LEFT JOIN request_interaction ra_i ON e.exhibition_id = ra_i.exhibition_id
GROUP BY e.exhibition_name
ORDER BY request_count DESC;"""

cursor.execute(query)
response = cursor.fetchall()
print(response)

[['Our Polluted World', 456], ['The Crenshaw Collection', 440], ['Thunder Lizards', 369], ['Adaptation', 194], ['Measureless to Man', 172], ['Cetacean Sensations', 131]]


**Count of Rating Values Across All Exhibitions**: Provides the count of each rating value (e.g., how many 'Amazing', 'Good', etc.) across all exhibitions.

In [81]:
query = """SELECT r.rating_description, COUNT(ra_i.rating_interaction_id) AS rating_count
FROM rating r
LEFT JOIN rating_interaction ra_i ON r.rating_id = ra_i.rating_id
GROUP BY r.rating_description
ORDER BY rating_count DESC;"""

cursor.execute(query)
response = cursor.fetchall()
print(response)

[['Neutral', 13222], ['Bad', 11533], ['Good', 9843], ['Terrible', 7995], ['Amazing', 7730]]


**Count of Request Types Across All Exhibitions**: Counts how many assistance and emergency requests were recorded across all exhibitions.

In [82]:
query = """SELECT req.request_description, COUNT(req_i.request_interaction_id) AS request_count
FROM request req
LEFT JOIN request_interaction req_i ON req.request_id = req_i.request_id
GROUP BY req.request_description
ORDER BY request_count DESC;"""

cursor.execute(query)
response = cursor.fetchall()
print(response)

[['assistance', 1694], ['emergency', 68]]


**Exhibition Interactions Summary (Rating and Request)**: Combines both rating and request interactions for each exhibition.

In [83]:
query = """SELECT e.exhibition_name,
       COUNT(DISTINCT ra_i.rating_interaction_id) AS total_ratings,
       COUNT(DISTINCT req_i.request_interaction_id) AS total_requests
FROM exhibition e
LEFT JOIN rating_interaction ra_i ON e.exhibition_id = ra_i.exhibition_id
LEFT JOIN request_interaction req_i ON e.exhibition_id = req_i.exhibition_id
GROUP BY e.exhibition_name
ORDER BY total_ratings DESC, total_requests DESC
LIMIT 1;"""





What exhibition is most frequently visited?

In [84]:
query = """SELECT e.exhibition_name, COUNT(ra_i.rating_interaction_id) AS visit_count
FROM exhibition e
JOIN rating_interaction ra_i ON e.exhibition_id = ra_i.exhibition_id
GROUP BY e.exhibition_name
ORDER BY visit_count DESC
LIMIT 1;"""

cursor.execute(query)
response = cursor.fetchall()
print(response)


[['Our Polluted World', 10205]]


What hour of the day has the most ratings?

In [85]:
query = """SELECT EXTRACT(HOUR FROM ra_i.event_at) AS hour_of_day, COUNT(ra_i.rating_interaction_id) AS rating_count
FROM rating_interaction ra_i
GROUP BY hour_of_day
ORDER BY rating_count DESC
LIMIT 1;"""

cursor.execute(query)
response = cursor.fetchall()
print(response)


[[Decimal('16'), 5911]]


What exhibition has the most emergencies?

In [86]:
query = """SELECT e.exhibition_name, COUNT(ri.request_interaction_id) AS emergency_count
FROM exhibition e
JOIN request_interaction ri ON e.exhibition_id = ri.exhibition_id
JOIN request r ON ri.request_id = r.request_id
WHERE r.request_value = 1  
GROUP BY e.exhibition_name
ORDER BY emergency_count DESC
LIMIT 1;"""

cursor.execute(query)
response = cursor.fetchall()
print(response)


[['Our Polluted World', 17]]


What is the average rating for each exhibition?

In [87]:
query = """SELECT e.exhibition_name, AVG(r.rating_value) AS avg_rating
FROM exhibition e
JOIN rating_interaction ri ON e.exhibition_id = ri.exhibition_id
JOIN rating r ON ri.rating_id = r.rating_id
GROUP BY e.exhibition_name
ORDER BY avg_rating DESC;"""

cursor.execute(query)
response = cursor.fetchall()
print(response)

[['Cetacean Sensations', Decimal('2.8809308857458120')], ['The Crenshaw Collection', Decimal('2.0509561433642505')], ['Measureless to Man', Decimal('1.9912783751493429')], ['Adaptation', Decimal('1.9831115660184237')], ['Thunder Lizards', Decimal('1.6631403118040089')], ['Our Polluted World', Decimal('1.13228809407153356198')]]


Count of rating interactions for each hour

In [88]:
query = """SELECT EXTRACT(HOUR FROM ra_i.event_at) AS hour_of_day, COUNT(ra_i.rating_interaction_id) AS rating_count
FROM rating_interaction ra_i
GROUP BY hour_of_day
ORDER BY rating_count DESC;"""

cursor.execute(query)
response = cursor.fetchall()
print(response)

[[Decimal('16'), 5911], [Decimal('14'), 5832], [Decimal('15'), 5826], [Decimal('13'), 5814], [Decimal('12'), 5781], [Decimal('11'), 5764], [Decimal('10'), 5750], [Decimal('9'), 5040], [Decimal('8'), 3781], [Decimal('17'), 819], [Decimal('7'), 3], [Decimal('18'), 2]]


In [89]:
query = """SELECT EXTRACT(HOUR FROM ri.event_at) AS hour_of_day, e.exhibition_name, ROUND(AVG(r.rating_value), 2) AS avg_rating
FROM exhibition e
JOIN rating_interaction ri ON e.exhibition_id = ri.exhibition_id
JOIN rating r ON ri.rating_id = r.rating_id
GROUP BY hour_of_day, e.exhibition_name
ORDER BY hour_of_day ASC;"""

cursor.execute(query)
response = cursor.fetchall()
print(response)

[[Decimal('7'), 'Measureless to Man', Decimal('1.00')], [Decimal('7'), 'The Crenshaw Collection', Decimal('4.00')], [Decimal('7'), 'Thunder Lizards', Decimal('0.00')], [Decimal('8'), 'Adaptation', Decimal('1.96')], [Decimal('8'), 'Cetacean Sensations', Decimal('2.83')], [Decimal('8'), 'Measureless to Man', Decimal('1.99')], [Decimal('8'), 'Our Polluted World', Decimal('1.16')], [Decimal('8'), 'The Crenshaw Collection', Decimal('2.03')], [Decimal('8'), 'Thunder Lizards', Decimal('1.66')], [Decimal('9'), 'Adaptation', Decimal('2.01')], [Decimal('9'), 'Cetacean Sensations', Decimal('2.91')], [Decimal('9'), 'Measureless to Man', Decimal('2.06')], [Decimal('9'), 'Our Polluted World', Decimal('1.19')], [Decimal('9'), 'The Crenshaw Collection', Decimal('2.04')], [Decimal('9'), 'Thunder Lizards', Decimal('1.68')], [Decimal('10'), 'Adaptation', Decimal('1.92')], [Decimal('10'), 'Cetacean Sensations', Decimal('2.89')], [Decimal('10'), 'Measureless to Man', Decimal('2.04')], [Decimal('10'), 'Our 

Are positive ratings more frequent before or after 1pm?

In [90]:
query = """SELECT COUNT(ri.rating_interaction_id) AS positive_rating_count_before_1pm
FROM rating_interaction ri
JOIN rating r ON ri.rating_id = r.rating_id
WHERE r.rating_value IN (3, 4)  -- Positive ratings
AND EXTRACT(HOUR FROM ri.event_at) < 13;"""

query = """SELECT COUNT(ri.rating_interaction_id) AS positive_rating_count_after_1pm
FROM rating_interaction ri
JOIN rating r ON ri.rating_id = r.rating_id
WHERE r.rating_value IN (3, 4)  -- Positive ratings
AND EXTRACT(HOUR FROM ri.event_at) >= 13;"""

cursor.execute(query)
response = cursor.fetchall()
print(response)

[[8544]]


Do Zoology exhibitions get better ratings than other types?

In [91]:
query = """SELECT AVG(r.rating_value) AS avg_rating_zoology
FROM exhibition e
JOIN rating_interaction ri ON e.exhibition_id = ri.exhibition_id
JOIN rating r ON ri.rating_id = r.rating_id
WHERE e.department_id = 4;"""

query = """SELECT AVG(r.rating_value) AS avg_rating_other_departments
FROM exhibition e
JOIN rating_interaction ri ON e.exhibition_id = ri.exhibition_id
JOIN rating r ON ri.rating_id = r.rating_id
WHERE e.department_id != 4;"""

cursor.execute(query)
response = cursor.fetchall()
print(response)


[[Decimal('1.6379937383384460')]]


# # Advanced queries

Which exhibitions receive fewer assistance requests than the average?

In [92]:
query = """WITH assistance_counts AS (
    SELECT e.exhibition_name, COUNT(r.request_value) AS count_assistance
    FROM exhibition e
    JOIN request_interaction ri ON e.exhibition_id = ri.exhibition_id
    JOIN request r ON ri.request_id = r.request_id
    WHERE r.request_value = 0
    GROUP BY e.exhibition_name
),
average_assistance AS (
    SELECT AVG(count_assistance) AS avg_assistance
    FROM assistance_counts)
SELECT ac.exhibition_name
FROM assistance_counts ac
JOIN average_assistance aa ON ac.count_assistance < aa.avg_assistance;"""

cursor.execute(query)
response = cursor.fetchall()
print(response)

[['Adaptation'], ['Cetacean Sensations'], ['Measureless to Man']]


Are there particular times when assistance requests/emergencies are more likely?

In [ ]:
query = """SELECT EXTRACT(HOUR FROM ri.event_at) AS hour_of_day, r.request_description, COUNT(ri.request_id) AS count_requests
FROM request_interaction ri
JOIN request r ON r.request_id = ri.request_id
GROUP BY hour_of_day, r.request_description
ORDER BY count_requests DESC;"""

cursor.execute(query)
response = cursor.fetchall()
print(response)

cursor.close()
conn.close()

Are emergencies more likely in exhibitions that are more exciting than others?

How does exhibition popularity change day-by-day?